In [179]:
# importation de pandas

from pandas import read_csv
import numpy as np

#insertion du fichier csv train dans un dataframe

df_train_full = read_csv( 'https://oml-data.s3.amazonaws.com/kaggle-give-me-credit-train.csv' )
df_test = read_csv('https://oml-data.s3.amazonaws.com/kaggle-give-me-credit-test.csv')

#affichage du fichier csv train full

#df_train_full.describe()

In [180]:
#change toutes les valeurs nan d'une colonne en 0

df_train_full['NumberOfDependents'] = df_train_full.fillna(0)

#change de type une colonne

df['age'] = df['age'].astype(int)

In [181]:
#création nouvelle colonne qui est une agrégation de 3 colonnes

df_train_full.insert( 6, 'NumberOfTimes30DaysOrLater', df_train_full['NumberOfTime30-59DaysPastDueNotWorse'] + 2*df_train_full['NumberOfTime60-89DaysPastDueNotWorse'] + 3*df_train_full['NumberOfTimes90DaysLate'])
df_test.insert( 6, 'NumberOfTimes30DaysOrLater', df_train_full['NumberOfTime30-59DaysPastDueNotWorse'] + 2*df_train_full['NumberOfTime60-89DaysPastDueNotWorse'] + 3*df_train_full['NumberOfTimes90DaysLate'])

In [182]:
#suppression des trois colonnes qui ont servit à l'agregation de la colonne précédente

del df_train_full['NumberOfTime30-59DaysPastDueNotWorse']
del df_train_full['NumberOfTimes90DaysLate']
del df_train_full['NumberOfTime60-89DaysPastDueNotWorse']

del df_test['NumberOfTime30-59DaysPastDueNotWorse']
del df_test['NumberOfTimes90DaysLate']
del df_test['NumberOfTime60-89DaysPastDueNotWorse']

In [183]:
#création de la colonne isOld

df_train_full.insert( 3, 'IsOld', 0)

df_test.insert( 3, 'IsOld', 0)

In [184]:
#remplacement des valeurs dans la colonnes isOld suivant les conditions préétablies

df_train_full.loc[df_train_full['age'] >= 70,'IsOld'] = 1

df_test.loc[df_train_full['age'] >= 70,'IsOld'] = 1

In [185]:
#suppression des valeurs anomalies

df_train_full.loc[df_train_full['NumberOfOpenCreditLinesAndLoans'] == 0,'NumberOfTimes30DaysOrLater'] = 0

#insertion de la colonne debt

df_train_full.insert( 6, 'Debt', 0)

df_test.insert( 6, 'Debt', 0)

#nous plaçons dans debt les valeurs anomalies de DebtRatio

def debtCalc(row):
    if row['DebtRatio'] > 1:
        row['Debt'] = row['DebtRatio']
        
    else:
        row['Debt'] = 12*row['DebtRatio']*row['MonthlyIncome']
    return row

for df in [df_train_full, df_test]:
    df['Debt'] = df[['Debt', 'DebtRatio', 'MonthlyIncome']].apply(debtCalc,axis=1)

#suppression des valeurs anomalies de la colonne debtRatio
df_train_full.loc[df_train_full['DebtRatio'] > 10,'DebtRatio'] = 0
df_test.loc[df_train_full['DebtRatio'] > 10,'DebtRatio'] = 0

In [186]:
#renomme la colonne 0

myList = list(df_train_full.columns)
myList[0] = 'Id'
df_train_full.columns = myList

myList = list(df_test.columns)
myList[0] = 'Id'
df_test.columns = myList

In [187]:
#affichage de vérifcation

df_train_full

,Id,RevolvingUtilizationOfUnsecuredLines,age,IsOld,DebtRatio,MonthlyIncome,NumberOfTimes30DaysOrLater,NumberOfOpenCreditLinesAndLoans,NumberRealEstateLoansOrLines,NumberOfDependents,SeriousDlqin2yrs
0,1,0.766127,45,0,0.802982,9120.0,2,13,6,1.0,1
1,2,0.957151,40,0,0.121876,2600.0,0,4,0,2.0,0
2,3,0.658180,38,0,0.085113,3042.0,4,2,0,3.0,0
3,4,0.233810,30,0,0.036050,3300.0,0,5,0,4.0,0
4,5,0.907239,49,0,0.024926,63588.0,1,7,1,5.0,0
...,...,...,...,...,...,...,...,...,...,...,...
149995,149996,0.040674,74,1,0.225131,2100.0,0,4,1,149996.0,0
149996,149997,0.299745,44,0,0.716562,5584.0,0,4,1,149997.0,0
149997,149998,0.246044,58,0,3870.000000,NaN,0,18,1,149998.0,0
149998,149999,0.000000,30,0,0.000000,5716.0,0,4,0,149999.0,0


In [188]:
df_train_full.to_csv('train_full.csv')
df_test.to_csv('test.csv')